# EDA

## Items to address

* Mixed Data Type - Column 10 Monthly Debt - cleaned up and changed to float
* Mixed Data Type - Column 16 Maximum Open Credit - cleaned up and changed to float
* NaN values
  * Credit Score ~ 19% --  will impute median for remaining items
  * Years in Current Job ~ 4% -- removed these rows
  * Annual Income ~ 19% -- will impure median for remaining items
  * Months Since Last Delinquent ~ 53% -- ,ore than 50% missing, removed column
  * Bankruptcies ~ less than 0.5% - removed, amount deminimis
  * Tax Liens ~ less than 0.5% - removed, amount deminimis
* Duplicates
  * All row duplicates
  * Duplicated loan ID values -- this is a unique identifier and while a customer ID can be tied to multiple loans, only one loan ID exists per loan
    * Many NaNs above can be handled by combining the duplicated loan ID rows (i.e row 1 and row 2 are duped loan IDs, row 1 is Nan columns A-C and row 2 is Nan columns D-H)
* Bad data
  * 'Other' and 'other' in 'purpose' column
  * 'Take a Trip' and 'vacation' repetitive in 'purpose' column
  * Unrealistic credit scores aka above 850
  * Current loan amounts at 999999999999 -- changed to 0, these values showed for fully paid loans (aka 0 balance)
  * Home Mortgage vs Own vs Have Mortgage - Consolidated Home and Have Mortgage, changed Own to Own Free
    


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
import numpy as np

In [3]:
loans_df = pd.read_csv('../../Loan-Dataset-Project/LoansDataset.csv')

# Note -- warning is generated that columns 10 and 16 have mixed data types, need to handle

C:\Users\nick_\AppData\Local\Temp\ipykernel_16336\2897786458.py:1: DtypeWarning: Columns (10,16) have mixed types. Specify dtype option on import or set low_memory=False.
  loans_df = pd.read_csv('../../Loan-Dataset-Project/LoansDataset.csv')


In [4]:
#Check for dupes and drop

loans_df.duplicated().sum()

# Drop 5,689 duplicated and check work

loans_nodupes = loans_df.drop_duplicates()
loans_nodupes.duplicated().sum()

np.int64(0)

In [5]:
print(id(loans_df))        # Memory address of loans_df
print(id(loans_nodupes))

3030943387328
3031721374288


In [6]:
# Big picture data check

loans_nodupes.info()


<class 'pandas.core.frame.DataFrame'>
Index: 105238 entries, 0 to 111106
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Loan ID                       105238 non-null  object 
 1   Customer ID                   105238 non-null  object 
 2   Loan Status                   105238 non-null  object 
 3   Current Loan Amount           105238 non-null  int64  
 4   Term                          105238 non-null  object 
 5   Credit Score                  83900 non-null   float64
 6   Years in current job          100788 non-null  object 
 7   Home Ownership                105238 non-null  object 
 8   Annual Income                 83900 non-null   float64
 9   Purpose                       105238 non-null  object 
 10  Monthly Debt                  105238 non-null  object 
 11  Years of Credit History       105238 non-null  float64
 12  Months since last delinquent  49003 non-null   fl

In [7]:
loans_nodupes.head()


,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,6cf51492-02a2-423e-b93d-676f05b9ad53,7c202b37-2add-44e8-9aea-d5b119aea935,Charged Off,12232,Short Term,7280.0,< 1 year,Rent,46643.0,Debt Consolidation,777.39,18.0,10.0,12,0,6762,7946,0.0,0.0
1,552e7ade-4292-4354-9ff9-c48031697d72,e7217b0a-07ac-47dd-b379-577b5a35b7c6,Charged Off,25014,Long Term,7330.0,10+ years,Home Mortgage,81099.0,Debt Consolidation,892.09,26.7,NaN,14,0,35706,77961,0.0,0.0
2,9b5e32b3-8d76-4801-afc8-d729d5a2e6b9,0a62fc41-16c8-40b5-92ff-9e4b763ce714,Charged Off,16117,Short Term,7240.0,9 years,Home Mortgage,60438.0,Home Improvements,1244.02,16.7,32.0,11,1,11275,14815,1.0,0.0
3,5419b7c7-ac11-4be2-a8a7-b131fb6d6dbe,30f36c59-5182-4482-8bbb-5b736849ae43,Charged Off,11716,Short Term,7400.0,3 years,Rent,34171.0,Debt Consolidation,990.94,10.0,NaN,21,0,7009,43533,0.0,0.0
4,1450910f-9495-4fc9-afaf-9bdf4b9821df,70c26012-bba5-42c0-8dcb-75295ada31bb,Charged Off,9789,Long Term,6860.0,10+ years,Home Mortgage,47003.0,Home Improvements,503.71,16.7,25.0,13,1,16913,19553,1.0,0.0


In [8]:
loans_nodupes.describe(include='all')

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
count,105238,105238,105238,1.052380e+05,105238,83900.000000,100788,105238,8.390000e+04,105238,105238,105238.000000,49003.000000,105238.000000,105238.000000,1.052380e+05,105238,105020.000000,105227.000000
unique,88910,88910,2,NaN,2,NaN,11,4,NaN,16,90362,NaN,NaN,NaN,NaN,NaN,67309,NaN,NaN
top,840e726a-4f01-450e-800e-c4b31fe1c49e,fd001b82-6a67-47f6-b64e-5129da842e83,Fully Paid,NaN,Short Term,NaN,10+ years,Home Mortgage,NaN,Debt Consolidation,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
freq,2,2,80065,NaN,76719,NaN,32663,51026,NaN,82847,55,NaN,NaN,NaN,NaN,NaN,450,NaN,NaN
mean,NaN,NaN,NaN,1.211641e+07,NaN,1102.051001,NaN,NaN,7.241759e+04,NaN,NaN,18.229131,34.918883,11.122551,0.166252,1.545967e+04,NaN,0.116464,0.029137
std,NaN,NaN,NaN,3.261291e+07,NaN,1521.347119,NaN,NaN,5.650503e+04,NaN,NaN,7.026107,21.959398,5.002925,0.482317,1.957319e+04,NaN,0.349179,0.261156
min,NaN,NaN,NaN,4.910000e+02,NaN,585.000000,NaN,NaN,4.033000e+03,NaN,NaN,3.600000,0.000000,0.000000,0.000000,0.000000e+00,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,8.174000e+03,NaN,707.000000,NaN,NaN,4.465800e+04,NaN,NaN,13.500000,16.000000,8.000000,0.000000,5.910250e+03,NaN,0.000000,0.000000
50%,NaN,NaN,NaN,1.421950e+04,NaN,726.000000,NaN,NaN,6.161400e+04,NaN,NaN,17.000000,32.000000,10.000000,0.000000,1.102550e+04,NaN,0.000000,0.000000
75%,NaN,NaN,NaN,2.400100e+04,NaN,741.000000,NaN,NaN,8.683400e+04,NaN,NaN,21.700000,51.000000,14.000000,0.000000,1.933150e+04,NaN,0.000000,0.000000


In [9]:
# Check NaN percentages

missing_percs = loans_nodupes.isna().sum().sort_values() / loans_nodupes.shape[0]
missing_percs

Loan ID                         0.000000
Customer ID                     0.000000
Loan Status                     0.000000
Current Loan Amount             0.000000
Term                            0.000000
Home Ownership                  0.000000
Monthly Debt                    0.000000
Purpose                         0.000000
Current Credit Balance          0.000000
Number of Credit Problems       0.000000
Years of Credit History         0.000000
Number of Open Accounts         0.000000
Maximum Open Credit             0.000000
Tax Liens                       0.000105
Bankruptcies                    0.002071
Years in current job            0.042285
Credit Score                    0.202759
Annual Income                   0.202759
Months since last delinquent    0.534360
dtype: float64

In [10]:
# Ttwo object rows that are mixed data types -- MaAx Open Credit and Monthly Debt

# Two rows have #VALUE! in Max Open Credit -- converted to Nan for time being so we can convert to float

loans_nodupes['Maximum Open Credit'] = loans_nodupes['Maximum Open Credit'].replace('#VALUE!', np.nan)


C:\Users\nick_\AppData\Local\Temp\ipykernel_16336\748279410.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans_nodupes['Maximum Open Credit'] = loans_nodupes['Maximum Open Credit'].replace('#VALUE!', np.nan)


In [11]:
loans_nodupes['Maximum Open Credit'] = loans_nodupes['Maximum Open Credit'].astype('float64')

C:\Users\nick_\AppData\Local\Temp\ipykernel_16336\2734657675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans_nodupes['Maximum Open Credit'] = loans_nodupes['Maximum Open Credit'].astype('float64')


In [12]:
loans_nodupes['Maximum Open Credit'].dtype

dtype('float64')

In [13]:
# Monthly Debt has $ signs that need to be removed before converting to float

loans_nodupes['Monthly Debt'] = loans_nodupes['Monthly Debt'].str.lstrip('$')

loans_nodupes['Monthly Debt'] = loans_nodupes['Monthly Debt'].astype('float64')


C:\Users\nick_\AppData\Local\Temp\ipykernel_16336\1032404361.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans_nodupes['Monthly Debt'] = loans_nodupes['Monthly Debt'].str.lstrip('$')
C:\Users\nick_\AppData\Local\Temp\ipykernel_16336\1032404361.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans_nodupes['Monthly Debt'] = loans_nodupes['Monthly Debt'].astype('float64')


In [14]:
loans_nodupes.info()


<class 'pandas.core.frame.DataFrame'>
Index: 105238 entries, 0 to 111106
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Loan ID                       105238 non-null  object 
 1   Customer ID                   105238 non-null  object 
 2   Loan Status                   105238 non-null  object 
 3   Current Loan Amount           105238 non-null  int64  
 4   Term                          105238 non-null  object 
 5   Credit Score                  83900 non-null   float64
 6   Years in current job          100788 non-null  object 
 7   Home Ownership                105238 non-null  object 
 8   Annual Income                 83900 non-null   float64
 9   Purpose                       105238 non-null  object 
 10  Monthly Debt                  65536 non-null   float64
 11  Years of Credit History       105238 non-null  float64
 12  Months since last delinquent  49003 non-null   fl

In [15]:
#Consolidate repetitive answers in Purpose Column

loans_nodupes['Purpose'] = loans_nodupes['Purpose'].str.replace('Other', 'other')
loans_nodupes['Purpose'] = loans_nodupes['Purpose'].str.replace('Take a Trip', 'vacation')

# #remove title case and replace spaces with _

loans_nodupes['Purpose'] = loans_nodupes['Purpose'].str.lower().str.replace(' ','_')


C:\Users\nick_\AppData\Local\Temp\ipykernel_16336\954497999.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans_nodupes['Purpose'] = loans_nodupes['Purpose'].str.replace('Other', 'other')
C:\Users\nick_\AppData\Local\Temp\ipykernel_16336\954497999.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans_nodupes['Purpose'] = loans_nodupes['Purpose'].str.replace('Take a Trip', 'vacation')
C:\Users\nick_\AppData\Local\Temp\ipykernel_16336\954497999.py:8: SettingWithCopyWarning: 
A value is trying to be 

In [16]:
loans_nodupes['Purpose'].value_counts()

Purpose
debt_consolidation      82847
other                    9610
home_improvements        6140
business_loan            1634
buy_a_car                1364
medical_bills            1184
buy_house                 703
vacation                  665
major_purchase            394
small_business            300
moving                    156
wedding                   122
educational_expenses      109
renewable_energy           10
Name: count, dtype: int64

In [17]:
# Change 999999999 in loan amount to NaN for future row combine

loans_nodupes['Current Loan Amount'] = loans_nodupes['Current Loan Amount'].replace(99999999, np.nan)

C:\Users\nick_\AppData\Local\Temp\ipykernel_16336\4072144335.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans_nodupes['Current Loan Amount'] = loans_nodupes['Current Loan Amount'].replace(99999999, np.nan)


In [18]:
(loans_nodupes['Current Loan Amount'] > 1000000).sum()

np.int64(0)

In [19]:
(loans_nodupes['Current Loan Amount'].isna()).sum()

np.int64(12738)

In [20]:
# HOme mortgage types are not clear

loans_nodupes['Home Ownership'].value_counts()

Home Ownership
Home Mortgage    51026
Rent             44402
Own Home          9601
HaveMortgage       209
Name: count, dtype: int64

In [21]:
# Updating to have only three values

loans_nodupes['Home Ownership'] = loans_nodupes['Home Ownership'].replace({'Home Mortgage': 'HaveMortgage', 'Own Home': 'OwnFree'})
loans_nodupes['Home Ownership'].value_counts()

C:\Users\nick_\AppData\Local\Temp\ipykernel_16336\4248085529.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans_nodupes['Home Ownership'] = loans_nodupes['Home Ownership'].replace({'Home Mortgage': 'HaveMortgage', 'Own Home': 'OwnFree'})


Home Ownership
HaveMortgage    51235
Rent            44402
OwnFree          9601
Name: count, dtype: int64

In [22]:
# # Loan ID and Customer ID are unique identifiers, but a customer can have more than one loan so loan # is truly unique in the dataframe. Check for dupes

loans_nodupes['Loan ID'].duplicated().sum()


np.int64(16328)

In [23]:

# # There are in fact duplicates in the Loan ID column. Filter to show all instances of a duplicate

loans_nodupes[loans_nodupes['Loan ID'].duplicated(keep=False)].sort_values(by='Loan ID')

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
32052,0004f37b-5859-40f6-98d0-367aa3b3f3f1,f662b062-5fa5-463d-b5c0-4e36d09fcab1,Fully Paid,13719.0,Short Term,NaN,1 year,OwnFree,NaN,home_improvements,777.38,13.6,2.0,12,0,6720,53335.0,0.0,0.0
97431,0004f37b-5859-40f6-98d0-367aa3b3f3f1,f662b062-5fa5-463d-b5c0-4e36d09fcab1,Fully Paid,13719.0,Short Term,724.0,1 year,OwnFree,34297.0,home_improvements,NaN,13.6,2.0,12,0,6720,53335.0,0.0,0.0
97878,00069ff1-a877-4d35-81be-7cd359b99956,ee3c27f2-766a-499c-9a96-d17c4d2faa16,Fully Paid,1433.0,Short Term,716.0,10+ years,HaveMortgage,108735.0,home_improvements,NaN,15.1,5.0,22,0,27706,49563.0,0.0,0.0
32218,00069ff1-a877-4d35-81be-7cd359b99956,ee3c27f2-766a-499c-9a96-d17c4d2faa16,Fully Paid,1433.0,Short Term,NaN,10+ years,HaveMortgage,NaN,home_improvements,2627.76,15.1,5.0,22,0,27706,49563.0,0.0,0.0
109095,000bc65a-6a7c-4566-86f3-203b4ec35eca,724bddb4-a23c-4759-ba6f-dc79c7dd5334,Fully Paid,29191.0,Short Term,715.0,2 years,Rent,92607.0,debt_consolidation,NaN,13.8,NaN,11,0,23473,33343.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102613,ffee9aee-ac81-4237-87d2-6b6e625139ad,f397c9b0-e7e4-4319-841e-973a46fff949,Fully Paid,6890.0,Short Term,701.0,1 year,Rent,35462.0,other,NaN,12.0,56.0,3,0,2881,3647.0,0.0,0.0
98510,fff5063a-9366-4659-bdf3-bf8c6fafa9b5,4b13e0aa-ec50-472b-b8f1-60ee584bbbc9,Fully Paid,5125.0,Short Term,722.0,3 years,Rent,14349.0,debt_consolidation,NaN,10.0,NaN,8,0,3457,12617.0,0.0,0.0
20860,fff5063a-9366-4659-bdf3-bf8c6fafa9b5,4b13e0aa-ec50-472b-b8f1-60ee584bbbc9,Fully Paid,5125.0,Short Term,NaN,3 years,Rent,NaN,debt_consolidation,74.85,10.0,NaN,8,0,3457,12617.0,0.0,0.0
90207,fffbc431-6a56-467c-a5f7-772928f96715,ae7c08b7-6b2e-4763-b148-035bf01c3072,Fully Paid,30327.0,Long Term,715.0,< 1 year,Rent,163376.0,debt_consolidation,NaN,13.9,NaN,8,0,9284,14644.0,0.0,0.0


In [24]:
loans_nodupes[loans_nodupes['Months since last delinquent'].isna()]['Loan Status'].value_counts()

Loan Status
Fully Paid     42631
Charged Off    13604
Name: count, dtype: int64

In [25]:
loans_nodupes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 105238 entries, 0 to 111106
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Loan ID                       105238 non-null  object 
 1   Customer ID                   105238 non-null  object 
 2   Loan Status                   105238 non-null  object 
 3   Current Loan Amount           92500 non-null   float64
 4   Term                          105238 non-null  object 
 5   Credit Score                  83900 non-null   float64
 6   Years in current job          100788 non-null  object 
 7   Home Ownership                105238 non-null  object 
 8   Annual Income                 83900 non-null   float64
 9   Purpose                       105238 non-null  object 
 10  Monthly Debt                  65536 non-null   float64
 11  Years of Credit History       105238 non-null  float64
 12  Months since last delinquent  49003 non-null   fl

In [26]:
# Take all rows with duplicate Loan ID and fill na values where possible

loans_noIDdupes = loans_nodupes.groupby('Loan ID', as_index=False).agg(lambda x: x.dropna().iloc[0] if not x.dropna().empty else None)

In [27]:
# There is a large jumo after 751 credit score to 5850, which is not a real credit score. Since 585 and 751 are real number, will drop 0 off the end of everything over 800


loans_noIDdupes.loc[loans_noIDdupes['Credit Score'] > 800, 'Credit Score'] = ((loans_nodupes.loc[loans_nodupes['Credit Score'] > 800, 'Credit Score'] // 10))

In [28]:
loans_noIDdupes.describe()


,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
count,80594.000000,68063.000000,7.282100e+04,65536.000000,88910.000000,40404.000000,88910.000000,88910.000000,8.891000e+04,8.890800e+04,88718.000000,88902.000000
mean,13984.078505,720.687378,7.240482e+04,962.406632,18.315313,35.080289,11.110348,0.160477,1.543593e+04,3.596531e+04,0.112672,0.028222
std,8291.958385,27.496034,5.771356e+04,632.799593,7.049175,22.008583,4.978461,0.473746,1.965873e+04,4.041805e+05,0.343300,0.257533
min,491.000000,585.000000,4.033000e+03,0.000000,3.600000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,7810.000000,709.000000,4.463800e+04,532.250000,13.500000,16.000000,8.000000,0.000000,5.949000e+03,1.277500e+04,0.000000,0.000000
50%,12058.000000,729.000000,6.158500e+04,843.770000,17.000000,32.000000,10.000000,0.000000,1.103900e+04,2.178100e+04,0.000000,0.000000
75%,19492.750000,741.000000,8.678500e+04,1251.342500,21.800000,51.000000,14.000000,0.000000,1.930100e+04,3.636700e+04,0.000000,0.000000
max,35875.000000,751.000000,8.713547e+06,12055.680000,70.500000,176.000000,76.000000,15.000000,1.730472e+06,6.998809e+07,7.000000,15.000000


In [29]:

loans_noIDdupes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88910 entries, 0 to 88909
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Loan ID                       88910 non-null  object 
 1   Customer ID                   88910 non-null  object 
 2   Loan Status                   88910 non-null  object 
 3   Current Loan Amount           80594 non-null  float64
 4   Term                          88910 non-null  object 
 5   Credit Score                  68063 non-null  float64
 6   Years in current job          85093 non-null  object 
 7   Home Ownership                88910 non-null  object 
 8   Annual Income                 72821 non-null  float64
 9   Purpose                       88910 non-null  object 
 10  Monthly Debt                  65536 non-null  float64
 11  Years of Credit History       88910 non-null  float64
 12  Months since last delinquent  40404 non-null  float64
 13  N

In [30]:
# Only columns showing null for current loan amount are paid off, therefore can be 0

loans_noIDdupes[loans_noIDdupes['Current Loan Amount'].isna()]['Loan Status'].value_counts()

Loan Status
Fully Paid    8316
Name: count, dtype: int64

In [31]:
loans_noIDdupes['Current Loan Amount'] = loans_noIDdupes['Current Loan Amount'].fillna(0)

In [32]:
loans_noIDdupes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88910 entries, 0 to 88909
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Loan ID                       88910 non-null  object 
 1   Customer ID                   88910 non-null  object 
 2   Loan Status                   88910 non-null  object 
 3   Current Loan Amount           88910 non-null  float64
 4   Term                          88910 non-null  object 
 5   Credit Score                  68063 non-null  float64
 6   Years in current job          85093 non-null  object 
 7   Home Ownership                88910 non-null  object 
 8   Annual Income                 72821 non-null  float64
 9   Purpose                       88910 non-null  object 
 10  Monthly Debt                  65536 non-null  float64
 11  Years of Credit History       88910 non-null  float64
 12  Months since last delinquent  40404 non-null  float64
 13  N

In [33]:
# Remove rows with insifnificant amounts of data missing

loans_noIDdupes = loans_noIDdupes[loans_noIDdupes['Bankruptcies'].notna()]
loans_noIDdupes = loans_noIDdupes[loans_noIDdupes['Tax Liens'].notna()]
loans_noIDdupes = loans_noIDdupes[loans_noIDdupes['Maximum Open Credit'].notna()]

In [34]:
# Check outstanding nan percentages

loans_noIDdupes.isna().sum().sort_values() / loans_noIDdupes.shape[0]


Loan ID                         0.000000
Customer ID                     0.000000
Loan Status                     0.000000
Current Loan Amount             0.000000
Term                            0.000000
Home Ownership                  0.000000
Years of Credit History         0.000000
Purpose                         0.000000
Current Credit Balance          0.000000
Number of Open Accounts         0.000000
Number of Credit Problems       0.000000
Maximum Open Credit             0.000000
Bankruptcies                    0.000000
Tax Liens                       0.000000
Years in current job            0.043025
Annual Income                   0.180926
Credit Score                    0.234445
Monthly Debt                    0.262974
Months since last delinquent    0.546745
dtype: float64

In [35]:
# Remove less than 5% from Years in current job

loans_noIDdupes = loans_noIDdupes[loans_noIDdupes['Years in current job'].notna()]

In [36]:
# Remove columsn iwht more than 50% missing

loans_for_splits = loans_noIDdupes.drop('Months since last delinquent', axis=1)

In [37]:
loans_for_splits.isna().sum().sort_values() / loans_for_splits.shape[0]

Loan ID                      0.000000
Customer ID                  0.000000
Loan Status                  0.000000
Current Loan Amount          0.000000
Term                         0.000000
Years in current job         0.000000
Home Ownership               0.000000
Purpose                      0.000000
Current Credit Balance       0.000000
Years of Credit History      0.000000
Number of Credit Problems    0.000000
Number of Open Accounts      0.000000
Maximum Open Credit          0.000000
Bankruptcies                 0.000000
Tax Liens                    0.000000
Annual Income                0.180638
Credit Score                 0.233607
Monthly Debt                 0.262700
dtype: float64

In [38]:
# Create Utilization columns

loans_for_splits['Credit Utilization'] = loans_for_splits['Current Credit Balance'] / loans_for_splits['Maximum Open Credit']


In [39]:
loans_for_splits[(loans_for_splits['Maximum Open Credit'] == 0) & (loans_for_splits['Current Credit Balance'] > 0)]

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Credit Utilization
120,0055afc1-ad71-44f8-ad20-d6dd125116ca,a09e6e85-a40d-462d-93c8-db0649554439,Fully Paid,14136.0,Short Term,NaN,4 years,Rent,NaN,debt_consolidation,4.11,11.8,6,0,4,0.0,0.0,0.0,inf
1113,033e996f-b88d-4c66-b275-0dd614b0c7be,993f047a-722f-4f90-9fa7-76b5c7069c52,Fully Paid,30549.0,Long Term,717.0,8 years,HaveMortgage,71281.0,debt_consolidation,466.30,13.0,10,0,1,0.0,0.0,0.0,inf
1188,037e7fb3-6c20-447d-8463-a66f289e7995,adf1bd1e-f49b-4844-84ef-8fc08c4c31f1,Fully Paid,12241.0,Short Term,NaN,10+ years,Rent,NaN,debt_consolidation,272.42,17.0,12,0,4,0.0,0.0,0.0,inf
3560,0a5bb105-1327-486e-9372-7baf17ddff08,341b16b9-0aa1-4717-8d48-eb1e85bdc509,Fully Paid,16230.0,Long Term,743.0,2 years,HaveMortgage,87238.0,debt_consolidation,131.59,20.5,6,0,21838,0.0,0.0,0.0,inf
3652,0a973ae7-7b44-4045-8c02-91f99dca2b80,2c75cbb7-38b9-4755-97c3-19efb1b19da8,Fully Paid,0.0,Short Term,748.0,10+ years,HaveMortgage,460971.0,home_improvements,679.93,20.6,10,0,145237,0.0,0.0,0.0,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80742,e830e04a-4847-4a32-868c-3e84acdbe131,fb7307e8-d195-4f6c-8efd-c6e6041dc058,Fully Paid,10137.0,Short Term,748.0,2 years,HaveMortgage,106439.0,wedding,879.89,23.0,6,0,5,0.0,0.0,0.0,inf
82434,ece5ca19-4fd3-42eb-92f2-367263658d1b,7b1ac682-aa4b-488e-bc98-8e0fb7468805,Fully Paid,35560.0,Short Term,746.0,1 year,Rent,177800.0,business_loan,1308.31,20.1,6,0,26020,0.0,0.0,0.0,inf
82969,ee9d2be3-503f-4024-90a4-3a3b8f0fda32,b771c494-c460-449f-8d0d-dc4455e03169,Fully Paid,14130.0,Short Term,725.0,10+ years,HaveMortgage,62531.0,debt_consolidation,NaN,23.4,7,0,1,0.0,0.0,0.0,inf
85378,f5af9e97-d2b6-49a3-88b2-efdf9b549e4a,ad7ccfd4-c419-48ab-8563-2f2aa98cb99b,Fully Paid,19674.0,Short Term,741.0,< 1 year,HaveMortgage,255762.0,debt_consolidation,NaN,15.0,8,0,127077,0.0,0.0,0.0,inf


In [40]:
#This created some null vales in new column, check why -- it is due to open credit being 0

loans_for_splits[loans_for_splits['Credit Utilization'].isna()]['Maximum Open Credit'].value_counts()


Maximum Open Credit
0.0    476
Name: count, dtype: int64

In [41]:
loans_for_splits['Credit Utilization'] = loans_for_splits['Credit Utilization'].fillna(0)
loans_for_splits[loans_for_splits['Credit Utilization'].isna()]

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Credit Utilization


In [42]:
loans_for_splits['Credit Utilization'] = np.where(np.isinf(loans_for_splits['Credit Utilization']), 0, loans_for_splits['Credit Utilization'])
loans_for_splits[np.isinf(loans_for_splits['Credit Utilization'])]

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Credit Utilization


In [43]:
loans_for_splits.to_csv('Loans_For_Analysis.csv')

In [44]:
loans_for_splits.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84899 entries, 0 to 88909
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Loan ID                    84899 non-null  object 
 1   Customer ID                84899 non-null  object 
 2   Loan Status                84899 non-null  object 
 3   Current Loan Amount        84899 non-null  float64
 4   Term                       84899 non-null  object 
 5   Credit Score               65066 non-null  float64
 6   Years in current job       84899 non-null  object 
 7   Home Ownership             84899 non-null  object 
 8   Annual Income              69563 non-null  float64
 9   Purpose                    84899 non-null  object 
 10  Monthly Debt               62596 non-null  float64
 11  Years of Credit History    84899 non-null  float64
 12  Number of Open Accounts    84899 non-null  int64  
 13  Number of Credit Problems  84899 non-null  int64  


In [45]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector

from category_encoders import HashingEncoder

from sklearn import set_config

set_config(transform_output='pandas')

X = loans_for_splits.drop(columns=['Loan Status', 'Loan ID', 'Customer ID'])
y = loans_for_splits['Loan Status'].copy()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

print(f"Train set size: {X_train.shape}")
print(f"Validation set size: {X_val.shape}")
print(f"Test set size: {X_test.shape}")

Train set size: (67919, 16)
Validation set size: (8490, 16)
Test set size: (8490, 16)


In [46]:
# Transformers

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
freq_imputer = SimpleImputer(strategy='most_frequent', add_indicator=True)
median_imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

In [47]:
num_cols = X_train.select_dtypes(include='number').columns
ohe_cols = X_train.select_dtypes(include='object').columns


In [48]:
ohe_pipe = Pipeline([
    ('Freq Imputer', freq_imputer),
    ('OHE', ohe)
    ])

num_pipe = Pipeline([
    ('Median Imputer', median_imputer),
    ('Standard Scaler', scaler)])

num_tuple = ('Numeric', num_pipe, num_cols)
ohe_tuple = ('OHE', ohe_pipe, ohe_cols)

In [49]:
preprocessor = ColumnTransformer([num_tuple, ohe_tuple],
                                  remainder='drop',verbose_feature_names_out=False)


preprocessor.fit(X_train)

X_train_proc = preprocessor.transform(X_train)

X_test_proc = preprocessor.transform(X_test)


In [50]:
X_train_proc['Debt to Income'] = (X_train_proc['Monthly Debt']) / (X_train_proc['Annual Income'] / 12)
X_train_proc


,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Credit Utilization,Term_Long Term,Term_Short Term,Years in current job_1 year,Years in current job_10+ years,Years in current job_2 years,Years in current job_3 years,Years in current job_4 years,Years in current job_5 years,Years in current job_6 years,Years in current job_7 years,Years in current job_8 years,Years in current job_9 years,Years in current job_< 1 year,Home Ownership_HaveMortgage,Home Ownership_OwnFree,Home Ownership_Rent,Purpose_business_loan,Purpose_buy_a_car,Purpose_buy_house,Purpose_debt_consolidation,Purpose_educational_expenses,Purpose_home_improvements,Purpose_major_purchase,Purpose_medical_bills,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding,Debt to Income
51391,-0.527159,-0.599742,-0.488836,-1.368577,-0.601622,-0.435308,-0.333785,-0.420099,-0.071259,-0.320541,-0.109587,0.832550,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.595989
40260,-0.778422,0.958878,0.470998,-0.296068,-0.367455,-0.435308,-0.333785,-0.526363,-0.069024,-0.320541,-0.109587,-0.252239,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.543175
25074,1.113332,-0.353644,-0.121899,0.371797,0.422858,-0.635870,-0.333785,-0.354428,-0.015305,-0.320541,-0.109587,-1.095555,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-36.600483
7318,-0.631161,0.261601,-0.406060,-0.813071,-1.670008,-0.635870,-0.333785,-0.468151,-0.059531,-0.320541,-0.109587,-0.411806,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.028104
24928,0.678274,0.958878,1.137587,1.948144,0.188691,0.166379,-0.333785,0.153971,-0.010458,-0.320541,-0.109587,0.144793,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.550287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6558,0.525745,0.876845,-0.355243,-1.181703,-0.118653,1.369753,-0.333785,-0.238002,-0.026121,-0.320541,-0.109587,-0.555159,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.917554
57514,1.212515,-1.173970,-0.321365,-0.207417,-1.260216,0.166379,1.818531,0.534031,0.040121,2.637448,-0.109587,-0.121254,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.745083
80438,-0.202154,-0.640758,-0.343371,-0.507237,-1.362664,-0.836433,-0.333785,-0.681631,-0.066025,-0.320541,-0.109587,-1.553939,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.726693
901,-0.290129,-0.066530,0.275332,-0.164292,-1.787092,-0.635870,-0.333785,0.452143,-0.025426,-0.320541,-0.109587,1.569341,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.160459


In [51]:
X_test_proc['Debt to Income'] = (X_test_proc['Monthly Debt']) / (X_test_proc['Annual Income'] / 12)
X_test_proc

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Credit Utilization,Term_Long Term,Term_Short Term,Years in current job_1 year,Years in current job_10+ years,Years in current job_2 years,Years in current job_3 years,Years in current job_4 years,Years in current job_5 years,Years in current job_6 years,Years in current job_7 years,Years in current job_8 years,Years in current job_9 years,Years in current job_< 1 year,Home Ownership_HaveMortgage,Home Ownership_OwnFree,Home Ownership_Rent,Purpose_business_loan,Purpose_buy_a_car,Purpose_buy_house,Purpose_debt_consolidation,Purpose_educational_expenses,Purpose_home_improvements,Purpose_major_purchase,Purpose_medical_bills,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding,Debt to Income
25858,-0.090307,0.425666,0.784747,2.064904,-0.030840,1.169190,-0.333785,-0.314335,-0.050898,-0.320541,-0.109587,-0.014914,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.575568
18106,2.560165,-1.953280,1.929588,0.928239,1.096088,0.166379,-0.333785,6.621902,0.485191,-0.320541,-0.109587,0.578683,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.772668
64548,-0.118549,0.630747,-0.056062,-0.431622,-0.689434,-0.836433,-0.333785,-0.388815,-0.065742,-0.320541,-0.109587,0.513095,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,92.388160
65189,-0.303914,-0.066530,-0.585402,-0.823834,-1.421206,-0.836433,-0.333785,-0.315486,-0.059556,-0.320541,-0.109587,0.492595,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16.887542
693,-1.434932,0.794813,-0.094301,1.324173,0.832650,-0.435308,-0.333785,-0.709261,-0.078574,-0.320541,-0.109587,-1.415030,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-168.503596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827,-0.666687,0.876845,-0.587103,-1.341918,-0.982143,-0.836433,-0.333785,-0.538026,-0.070265,-0.320541,-0.109587,-0.256572,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.427907
2219,-0.705463,0.261601,-0.155089,-0.409546,-1.069955,-0.234746,-0.333785,-0.224688,-0.057147,-0.320541,-0.109587,0.832437,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.688529
66696,0.849293,0.261601,-0.155089,-0.900127,1.300984,-0.836433,1.818531,0.084296,-0.036133,2.637448,-0.109587,0.897985,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.647139
39512,0.396304,0.630747,-0.431866,0.086939,-0.367455,0.567503,-0.333785,-0.255321,-0.063869,-0.320541,-0.109587,1.249839,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.415725
